Let's do a quick example comparing `jitr` to the standard Runge-Kutta ODE solver implemented in `scipy.integrate.solve_ivp`:

In [22]:
import numpy as np
from scipy.integrate import solve_ivp
from numba import njit
from jitr import reactions, rmatrix, utils
from jitr.reactions.potentials import woods_saxon_potential

First let's check our library versions and configs, as this will affect performance:

In [14]:
import scipy as sc

sc.__version__

'1.11.4'

In [15]:
import numba

numba.__version__

'0.58.1'

In [16]:
np.__version__

'1.26.4'

In [17]:
np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /home/kyle/mambaforge/envs/om/include
    lib directory: /home/kyle/mambaforge/envs/om/lib
    name: blas
    openblas configuration: unknown
    pc file directory: /home/kyle/mambaforge/envs/om/lib/pkgconfig
    version: 3.9.0
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep140371046681712
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -march=nocona, -mtune=haswell, -ftree-vectorize, -fPIC, -fstack-protector-strong,
      -fno-plt, -O2, -ffunction-sections, -pipe, -isystem, /home/kyle/mambaforge/envs/om/include,
      -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/numpy_1707225380409/work=/usr/local/src/conda/numpy-1.26.4,
      -fdebug-prefix-map=/home/kyle/mambaforge/envs/om=/usr/local/src/conda-prefix,
      -DNDEBUG, -D_FORTI

Great, now let's set up our system and solver with `jitr`:

In [21]:
sys = reactions.ProjectileTargetSystem(
    channel_radii=np.array([5 * (2 * np.pi)]),
    l=np.array([0]),
    mass_target=44657,
    mass_projectile=938.3,
    Ztarget=40,
    Zproj=0,
    nchannels=1,
)

# COM frame energy
Elab = 14.1

# Lagrange-Mesh R-matrix solver
solver = rmatrix.Solver(40)

# channels holds info for the elastic scattering channel
Elab = 42.1
mu, Ecm, k, eta = utils.kinematics.classical_kinematics(
    sys.mass_target, sys.mass_projectile, Elab, sys.Zproj * sys.Ztarget
)
channels, asymptotics = sys.coupled(Ecm, mu, k, eta)

In [19]:
# Woods-Saxon potential parameters
V0 = 60  # real potential strength
W0 = 20  # imag potential strength
R0 = 4  # Woods-Saxon potential radius
a0 = 0.5  # Woods-Saxon potential diffuseness
params = (V0, W0, R0, a0)

In [23]:
# run solver
R, S, uext_prime_boundary = solver.solve(
    channels, asymptotics, woods_saxon_potential, params
)

/home/kyle/umich/jitr/src/jitr/rmatrix/rmatrix_solver.py:44: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(complex128, 1, 'C', False, aligned=True), Array(complex128, 2, 'A', False, aligned=True))
  R, Ainv = rmatrix_with_inverse(A, b, nchannels, nbasis, a)


In [25]:
print(S[0, 0])

(-1.897689085230767+1.673005014393317j)


Great, now let's use `scipy` and see if we get the same $\mathcal{S}$-matrix:

In [29]:
# Runge-Kutta
from jitr.utils import schrodinger_eqn_ivp_order1

channel_data_rk = reactions.make_channel_data(channels)
domain, init_con = channel_data_rk[0].initial_conditions()
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, channel_data_rk[0], woods_saxon_potential, params
    ),
    domain,
    init_con,
    dense_output=True,
    atol=1.0e-7,
    rtol=1.0e-7,
).sol

In [32]:
a = channel_data_rk[0].domain[1]
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = jitr.utils.smatrix(R_rk, a, channel_data_rk[0].l, channel_data_rk[0].eta)
print(S_rk)

(-1.8975431198137418+1.6726981929138467j)


In [33]:
100 * (S[0, 0] - S_rk) / S_rk  # percent difference in real and imag parts of S

(0.012349525657186439-0.005283209953037379j)

Great, our solvers agree up to high precision. Now let's compare the runtime of the two solver options:

In [37]:
%%timeit
R, S, uext_prime_boundary = solver.solve(
    channels, asymptotics, woods_saxon_potential, params
)

309 µs ± 3.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [38]:
%%timeit
domain, init_con = channel_data_rk[0].initial_conditions()
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, channel_data_rk[0], woods_saxon_potential, params
    ),
    domain,
    init_con,
    dense_output=True,
    atol=1.0e-7,
    rtol=1.0e-7,
).sol
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = jitr.utils.smatrix(R_rk, a, channel_data_rk[0].l, channel_data_rk[0].eta)

78.8 ms ± 581 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


On my machine `jitr` is faster by about 250 times!

(This does, of course, depend on the solver paramaters; `atol` and `rtol` for `solve_ivp`, and `nbasis` for `LagrangeRMatrixSolver` ).